<a href="https://colab.research.google.com/github/Duke-BC-DL-for-Energy-Infrastructure/Colab-Notebooks-for-Training-Models/blob/main/Ratio%20Test%20New%20Background%20Imagery%20%2B%20New%20Syn%20Gen/Train_NE_Val_NW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import glob
import torch
import os

from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
!git clone https://github.com/Duke-BC-DL-for-Energy-Infrastructure/yolov3.git  # clone our fork of the ultralytics repo

In [ ]:
!mkdir -p data/

# get our data and unzip it
!wget -nc -O data/images.zip https://duke.box.com/shared/static/rttn4ujt4h9sbuba4poibdvtlqr7fhvz.zip
!wget -nc -O data/labels.zip https://duke.box.com/shared/static/psea4em0oh1ng99neowslj9hfz79sckn.zip

!unzip -n -q data/images.zip -d data
!unzip -n -q data/labels.zip -d data

# get the synthetic images and labels and unzip those into the same directory
!wget -nc -O data/synthetic_images.zip https://duke.box.com/shared/static/xtjkln8gb3zlt4qefhkjp8lv5q4m6u4a.zip
!wget -nc -O data/synthetic_labels.zip https://duke.box.com/shared/static/ks0epms14ql3f3ix318vsa5rn8hwh1hx.zip

!unzip -n -q data/synthetic_images.zip -d data
!unzip -n -q data/synthetic_labels.zip -d data

In [ ]:
# get modified cfg file that works for 1 class
!rm -rf yolov3/cfg 

!wget -nc -O yolov3/cfg.zip https://duke.box.com/shared/static/pq52dolw5lo5ds8fd7g9v72ekzlngekz.zip
!unzip -n -q yolov3/cfg.zip -d yolov3

In [ ]:
# GET CROSS DOMAIN DATA
# delete the original data folder so we can unzip our own data folder with the wind turbine data
!rm -rf yolov3/data

# load in the data.zip file from the box
!wget -nc -O yolov3/baseline.zip https://duke.box.com/shared/static/f4kme2valirs8hyzhrym4duem5dddrod.zip
!unzip -n -q yolov3/baseline.zip -d yolov3

# get the synthetic_data file from the box
!wget -nc -O yolov3/adding_synthetic_50.zip https://duke.box.com/shared/static/u9ksdi6ynz3xnnv31ykxvusp75q2d0f8.zip
!unzip -n -q yolov3/adding_synthetic_50.zip -d yolov3

!wget -nc -O yolov3/adding_synthetic_100.zip https://duke.box.com/shared/static/ltsb64mt3p91fftshgaelxlbonrwg52a.zip
!unzip -n -q yolov3/adding_synthetic_100.zip -d yolov3

!wget -nc -O yolov3/adding_synthetic_200.zip https://duke.box.com/shared/static/owwf6hiprpwsdno4ji9bhgcadlaq9teh.zip
!unzip -n -q yolov3/adding_synthetic_200.zip -d yolov3

In [ ]:
%pwd

In [ ]:
%cd yolov3

In [ ]:
# Training on NE and validating on NW
!python3 train.py --cfg cfg/yolov3-spp.cfg --data baseline/baseline.data --img-size 608 --epochs 300 --batch-size 10 --device 0

In [ ]:
!python3 test.py --cfg cfg/yolov3-spp.cfg --data baseline/baseline.data --img-size 608 --weights weights/last.pt --conf-thres 0.0001

In [ ]:
# Training on NE + 50 synthetic NW and validating on NW
!python3 train.py --cfg cfg/yolov3-spp.cfg --data adding_synthetic_50/adding_synthetic.data --img-size 608 --epochs 300 --batch-size 10 --device 0

In [ ]:
!python3 test.py --cfg cfg/yolov3-spp.cfg --data adding_synthetic_50/adding_synthetic.data --img-size 608 --weights weights/last.pt --conf-thres 0.0001

In [ ]:
# Training on NE + 100 synthetic NW and validating on NW
!python3 train.py --cfg cfg/yolov3-spp.cfg --data adding_synthetic_100/adding_synthetic.data --img-size 608 --epochs 300 --batch-size 10 --device 0

In [ ]:
!python3 test.py --cfg cfg/yolov3-spp.cfg --data adding_synthetic_100/adding_synthetic.data --img-size 608 --weights weights/last.pt --conf-thres 0.0001

In [ ]:
# Training on NE + 200 synthetic NW and validating on NW
!python3 train.py --cfg cfg/yolov3-spp.cfg --data adding_synthetic_200/adding_synthetic.data --img-size 608 --epochs 300 --batch-size 10 --device 0

In [ ]:
!python3 test.py --cfg cfg/yolov3-spp.cfg --data adding_synthetic_200/adding_synthetic.data --img-size 608 --weights weights/last.pt --conf-thres 0.0001

In [ ]:
# Create folder for validation dataset, so that we can run detect.py on it

VAL_PATH = 'baseline/val_img_paths.txt' # Provide path to .txt file for validation set

from shutil import copyfile
%mkdir ../data/val

with open(VAL_PATH, 'r') as f:
  val_imgs =  [path.replace('\n', '') for path in f.readlines()]

for img in val_imgs:
  # copy image to val folder
  copyfile(img, img.replace('images', 'val'))

In [ ]:
# DETECT using last.pt
!python3 detect.py --names baseline/wnd.names --weights weights/last.pt --source ../data/val --img-size 608